In [15]:
from collections import Counter
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_csv("../../data/night/night_dataset.csv")
df_1 = df.drop('title', axis=1)
df_1 = df_1.drop('abstract', axis=1)
df = df_1.drop('claim', axis=1)

x = df.loc[:, df.columns!='valid_patent'].astype(str)
y = df['valid_patent'].astype(int)

## Train/Test Split

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=50)

## SMOTE

In [18]:
print('Original dataset shape {}'.format(Counter(y_train)))
sm = SMOTE(random_state=42)
x_res , y_res = sm.fit_sample(x_train, y_train)
print('Resampled dataset shape {}'.format(Counter(y_res)))
print('Y test dataset shpae {}'.format(Counter(y_test)))

Original dataset shape Counter({0: 773, 1: 92})
Resampled dataset shape Counter({0: 773, 1: 773})
Y test dataset shpae Counter({0: 336, 1: 35})


## Model

### Logistic Regression

In [19]:
from sklearn import linear_model, metrics
from sklearn.metrics import classification_report, confusion_matrix

In [20]:
logreg = linear_model.LogisticRegression(fit_intercept=False)
logreg.fit(x_res, y_res)

result = logreg.predict(x_test)

print("This classification report of using threshold: \n",classification_report(y_test, result))
print ("This is accuracy score:",metrics.accuracy_score(y_test, result),"\n")

This classification report of using threshold: 
              precision    recall  f1-score   support

          0       0.92      0.79      0.85       336
          1       0.14      0.31      0.19        35

avg / total       0.84      0.75      0.79       371

This is accuracy score: 0.7493261455525606 



In [21]:
prob = logreg.predict_proba(x_test)

y_pred = np.zeros(len(x_test))
y_pred[prob[:,1] > 0.35] = 1

print("This classification report of using threshold: \n",classification_report(y_test, y_pred))
print ("This is accuracy score:",metrics.accuracy_score(y_test, y_pred),"\n")

This classification report of using threshold: 
              precision    recall  f1-score   support

          0       0.95      0.24      0.38       336
          1       0.11      0.89      0.19        35

avg / total       0.87      0.30      0.36       371

This is accuracy score: 0.2991913746630728 



### SVM

In [22]:
from sklearn.linear_model import SGDClassifier
from sklearn import svm

svm = svm.LinearSVC()
svm.fit(x_res, y_res)

result = svm.predict(x_test)

print("This classification report of using threshold: \n",classification_report(y_test, result))
print ("This is accuracy score:",metrics.accuracy_score(y_test, result),"\n")

This classification report of using threshold: 
              precision    recall  f1-score   support

          0       0.89      0.76      0.82       336
          1       0.05      0.11      0.07        35

avg / total       0.81      0.70      0.75       371

This is accuracy score: 0.7008086253369272 



In [23]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

svc = CalibratedClassifierCV(base_estimator=LinearSVC(penalty='l2', dual=False), cv=5)
svc.fit(x_res, y_res)


result = svc.predict(x_test)

print("This classification report of using threshold: \n",classification_report(y_test, result))
print ("This is accuracy score:",metrics.accuracy_score(y_test, result),"\n")

This classification report of using threshold: 
              precision    recall  f1-score   support

          0       0.92      0.79      0.85       336
          1       0.14      0.31      0.19        35

avg / total       0.84      0.75      0.79       371

This is accuracy score: 0.7493261455525606 



In [24]:
prob = svc.predict_proba(x_test)

y_pred = np.zeros(len(x_test))
y_pred[prob[:,-1] > 0.75] = 1

print("This classification report of using threshold: \n",classification_report(y_test, y_pred))
print ("This is accuracy score:",metrics.accuracy_score(y_test, y_pred),"\n")

This classification report of using threshold: 
              precision    recall  f1-score   support

          0       0.91      1.00      0.95       336
          1       0.00      0.00      0.00        35

avg / total       0.82      0.91      0.86       371

This is accuracy score: 0.9056603773584906 



/home/yehoon/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Decision Tree

In [48]:
from sklearn import tree

dt = tree.DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_leaf=5)
dt.fit(x_res, y_res)

result = dt.predict(x_test)

print("This classification report of using threshold: \n",classification_report(y_test, result))
print ("This is accuracy score:",metrics.accuracy_score(y_test, result),"\n")

This classification report of using threshold: 
              precision    recall  f1-score   support

          0       0.99      0.99      0.99       565
          1       0.36      0.57      0.44         7

avg / total       0.99      0.98      0.98       572

This is accuracy score: 0.9825174825174825 



In [49]:
prob = dt.predict_proba(x_test)

y_pred = np.zeros(len(x_test))
y_pred[prob[:,-1] > 0.75] = 1

print("This classification report of using threshold: \n",classification_report(y_test, y_pred))
print ("This is accuracy score:",metrics.accuracy_score(y_test, y_pred),"\n")

This classification report of using threshold: 
              precision    recall  f1-score   support

          0       0.99      0.99      0.99       565
          1       0.44      0.57      0.50         7

avg / total       0.99      0.99      0.99       572

This is accuracy score: 0.986013986013986 



### Random Forest

In [50]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=5)
rf.fit(x_res, y_res)

result = rf.predict(x_test)

print("This classification report of using threshold: \n",classification_report(y_test, result))
print ("This is accuracy score:",metrics.accuracy_score(y_test, result),"\n")

This classification report of using threshold: 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00       565
          1       0.75      0.86      0.80         7

avg / total       1.00      0.99      0.99       572

This is accuracy score: 0.9947552447552448 



In [51]:
prob = rf.predict_proba(x_test)

y_pred = np.zeros(len(x_test))
y_pred[prob[:,-1] > 0.8] = 1

print("This classification report of using threshold: \n",classification_report(y_test, y_pred))
print ("This is accuracy score:",metrics.accuracy_score(y_test, y_pred),"\n")

This classification report of using threshold: 
              precision    recall  f1-score   support

          0       0.99      1.00      1.00       565
          1       1.00      0.43      0.60         7

avg / total       0.99      0.99      0.99       572

This is accuracy score: 0.993006993006993 



In [52]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(learning_rate_init=0.01, activation='relu', solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(100, ), random_state=1)
mlp.fit(x_res, y_res)

result = mlp.predict(x_test)

print("This classification report of using threshold: \n",classification_report(y_test, result))
print ("This is accuracy score:",metrics.accuracy_score(y_test, result),"\n")

This classification report of using threshold: 
              precision    recall  f1-score   support

          0       0.99      0.89      0.94       565
          1       0.02      0.14      0.03         7

avg / total       0.98      0.88      0.92       572

This is accuracy score: 0.8793706293706294 

